In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,54.79,64,91,16.71,overcast clouds
1,1,Mayo,US,38.8876,-76.5119,72.36,91,100,3.44,overcast clouds
2,2,Shingu,JP,33.7333,135.9833,78.40,87,81,1.32,broken clouds
3,3,Atuona,PF,-9.8000,-139.0333,77.86,77,23,18.01,few clouds
4,4,Balakovo,RU,52.0278,47.8007,66.40,66,51,5.14,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Shingu,JP,33.7333,135.9833,78.40,87,81,1.32,broken clouds
3,3,Atuona,PF,-9.8000,-139.0333,77.86,77,23,18.01,few clouds
5,5,Kishni,IN,27.0247,79.2634,88.66,57,100,10.11,overcast clouds
6,6,Ajdabiya,LY,30.7554,20.2263,77.47,70,0,12.46,clear sky
8,8,Victoria,HK,22.2855,114.1577,87.96,85,14,5.99,few clouds
...,...,...,...,...,...,...,...,...,...,...
512,512,Asilah,MA,35.4650,-6.0348,84.16,68,0,17.54,clear sky
518,518,Sohag,EG,26.5570,31.6948,85.41,24,35,10.56,scattered clouds
520,520,Naples,IT,40.8333,14.2500,76.75,47,0,6.91,clear sky
525,525,Patiya,BD,22.2931,91.9762,76.80,93,52,4.25,broken clouds


In [12]:
# 4a. Determine if there are any empty rows
preferred_cities_df.count()

City_ID       201
City          201
Lat           201
Lng           201
Max Temp      201
Humidity      201
Cloudiness    201
Wind Speed    201
Country       201
Date          201
dtype: int64

In [29]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")

In [30]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_clean_df = preferred_cities_df

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
# Populate None as the value under "Hotel Name" column initially
hotel_df["Hotel Name"] = None
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Shingu,JP,78.40,broken clouds,33.7333,135.9833,None
3,Atuona,PF,77.86,few clouds,-9.8000,-139.0333,None
5,Kishni,IN,88.66,overcast clouds,27.0247,79.2634,None
6,Ajdabiya,LY,77.47,clear sky,30.7554,20.2263,None
8,Victoria,HK,87.96,few clouds,22.2855,114.1577,None
...,...,...,...,...,...,...,...
512,Asilah,MA,84.16,clear sky,35.4650,-6.0348,None
518,Sohag,EG,85.41,scattered clouds,26.5570,31.6948,None
520,Naples,IT,76.75,clear sky,40.8333,14.2500,None
525,Patiya,BD,76.80,broken clouds,22.2931,91.9762,None


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")       

Hotel not found... skipping.57
Hotel not found... skipping.153
Hotel not found... skipping.290
Hotel not found... skipping.326
Hotel not found... skipping.361
Hotel not found... skipping.389
Hotel not found... skipping.400
Hotel not found... skipping.419
Hotel not found... skipping.432
Hotel not found... skipping.506


In [34]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Shingu,JP,78.40,broken clouds,33.7333,135.9833,Shingu UI Hotel
3,Atuona,PF,77.86,few clouds,-9.8000,-139.0333,Villa Enata
5,Kishni,IN,88.66,overcast clouds,27.0247,79.2634,श्री आंनद ग्रुप
6,Ajdabiya,LY,77.47,clear sky,30.7554,20.2263,Almadina Hotel
8,Victoria,HK,87.96,few clouds,22.2855,114.1577,Mini Central
...,...,...,...,...,...,...,...
512,Asilah,MA,84.16,clear sky,35.4650,-6.0348,Hotel Zelis
518,Sohag,EG,85.41,scattered clouds,26.5570,31.6948,Civil Registry - Sohag 1
520,Naples,IT,76.75,clear sky,40.8333,14.2500,Renaissance Naples Hotel Mediterraneo
525,Patiya,BD,76.80,broken clouds,22.2931,91.9762,Sorry


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))